In [1]:
from selenium import webdriver
import time
from bs4 import BeautifulSoup
import requests
from tqdm import tqdm
from selenium.webdriver.common.keys import Keys
import pandas as pd
import math
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import ElementNotInteractableException
from selenium.common.exceptions import StaleElementReferenceException
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
import numpy as np
import warnings
warnings.filterwarnings("ignore")
import json

In [2]:
data = pd.read_csv('C:/Users/정예림/Desktop/추천시스템 프로젝트/data/line9.csv')
data
# 카페 데이터만
data = data.loc[data['상권업종중분류명'] == '커피점/카페']  
data

,Unnamed: 0,상가업소번호,상호명,지점명,상권업종대분류코드,상권업종대분류명,상권업종중분류코드,상권업종중분류명,상권업종소분류코드,상권업종소분류명,...,구우편번호,신우편번호,동정보,층정보,호정보,경도,위도,역명,거리,카카오url
7,66,28524822,빈트리망원점,망원점,Q,음식,Q12,커피점/카페,Q12A01,커피전문점/카페/다방,...,121230,4007.0,NaN,1,NaN,126.899339,37.556041,9호선 선유도,1294.649872,NaN
13,137,15520570,더블유카페,역삼2호점,Q,음식,Q12,커피점/카페,Q12A01,커피전문점/카페/다방,...,135921,6225.0,NaN,1,2,127.043902,37.499426,9호선 선정릉,764.611819,NaN
15,147,22646116,커피빈,코리아선릉역점,Q,음식,Q12,커피점/카페,Q12A01,커피전문점/카페/다방,...,135080,6150.0,NaN,1,NaN,127.048223,37.504750,9호선 삼성중앙,461.237983,NaN
29,222,25437483,카페그리다,NaN,Q,음식,Q12,커피점/카페,Q12A01,커피전문점/카페/다방,...,135090,6159.0,NaN,1,NaN,127.052597,37.506767,9호선 삼성중앙,554.451907,NaN
35,274,23739181,홈플러스어라운드홈,강서점,Q,음식,Q12,커피점/카페,Q12A01,커피전문점/카페/다방,...,157030,7567.0,NaN,1,NaN,126.854800,37.558456,9호선 가양,236.335955,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12774,135527,17249536,티컬렉티브,삼성점,Q,음식,Q12,커피점/카페,Q12A01,커피전문점/카페/다방,...,135868,6097.0,NaN,NaN,NaN,127.048787,37.511981,9호선 삼성중앙,35.645121,NaN
12782,135625,17273431,트윗젤,NaN,Q,음식,Q12,커피점/카페,Q12A01,커피전문점/카페/다방,...,135876,6212.0,220,NaN,116,127.048927,37.504494,9호선 삼성중앙,478.709107,NaN
12784,135632,17254243,후르츠13.5,NaN,Q,음식,Q12,커피점/카페,Q12A01,커피전문점/카페/다방,...,157805,7788.0,NaN,12,NaN,126.825403,37.568078,9호선 마곡나루,235.823443,NaN
12785,135633,17252972,카페,MJ보드,Q,음식,Q12,커피점/카페,Q12A01,커피전문점/카페/다방,...,157805,7788.0,NaN,13,NaN,126.827682,37.568569,9호선 마곡나루,135.140328,NaN


In [3]:
data = data[["상호명","상권업종중분류명","상권업종소분류명","도로명주소","역명","거리","카카오url"]]
data["역"] = data["역명"].apply(lambda x : x.split(" ")[1])
data["역"] = data["역"].apply(lambda x : x.split("(")[0])
data["검색어"] = data.apply(lambda x : x["역"] + "역 " + str(x["상호명"]),axis=1)
data

,상호명,상권업종중분류명,상권업종소분류명,도로명주소,역명,거리,카카오url,역,검색어
7,빈트리망원점,커피점/카페,커피전문점/카페/다방,서울특별시 마포구 망원로 22,9호선 선유도,1294.649872,NaN,선유도,선유도역 빈트리망원점
13,더블유카페,커피점/카페,커피전문점/카페/다방,서울특별시 강남구 언주로 407,9호선 선정릉,764.611819,NaN,선정릉,선정릉역 더블유카페
15,커피빈,커피점/카페,커피전문점/카페/다방,서울특별시 강남구 테헤란로 335,9호선 삼성중앙,461.237983,NaN,삼성중앙,삼성중앙역 커피빈
29,카페그리다,커피점/카페,커피전문점/카페/다방,서울특별시 강남구 테헤란로69길 14,9호선 삼성중앙,554.451907,NaN,삼성중앙,삼성중앙역 카페그리다
35,홈플러스어라운드홈,커피점/카페,커피전문점/카페/다방,서울특별시 강서구 화곡로 398,9호선 가양,236.335955,NaN,가양,가양역 홈플러스어라운드홈
...,...,...,...,...,...,...,...,...,...
12774,티컬렉티브,커피점/카페,커피전문점/카페/다방,서울특별시 강남구 봉은사로 449,9호선 삼성중앙,35.645121,NaN,삼성중앙,삼성중앙역 티컬렉티브
12782,트윗젤,커피점/카페,커피전문점/카페/다방,서울특별시 강남구 테헤란로 340,9호선 삼성중앙,478.709107,NaN,삼성중앙,삼성중앙역 트윗젤
12784,후르츠13.5,커피점/카페,커피전문점/카페/다방,서울특별시 강서구 마곡서로 158,9호선 마곡나루,235.823443,NaN,마곡나루,마곡나루역 후르츠13.5
12785,카페,커피점/카페,커피전문점/카페/다방,서울특별시 강서구 마곡중앙로 165,9호선 마곡나루,135.140328,NaN,마곡나루,마곡나루역 카페


In [4]:
items = data["검색어"][1025:]
items

5933     사평역 간단하지만특별한화피디Cafe
5941            언주역 씨제이프레시웨이
5945              삼성중앙역 르플로우
5946              신논현역 바나프레소
5951          고속터미널역 청년커피LAB
                ...         
12774            삼성중앙역 티컬렉티브
12782              삼성중앙역 트윗젤
12784          마곡나루역 후르츠13.5
12785               마곡나루역 카페
12799              노량진역 카페시모
Name: 검색어, Length: 1190, dtype: object

In [5]:
rating_df = pd.DataFrame(columns = ["UserID","ItemID","Rating","Timestamp"])
img_elem = pd.DataFrame(columns = ["ItemID","img_url"])
review_elem = pd.DataFrame(columns = ["ItemID", "review"])
user_review_id = {}
review_json = {}
image_json = {}

In [6]:
# 드라이버 구동
driver = webdriver.Chrome(executable_path='C:/Users/정예림/Desktop/추천시스템 프로젝트/chromedriver.exe')
# 창 크기 최대화
driver.maximize_window() 



count = 0
current = 0
indexing = 0
goal = len(items)
for item in items :
    current += 1
    print('진행상황 : ', current,'/',goal,sep="")
    # 리뷰가 없을 때의 코드
    driver.get("https://map.kakao.com/") # 카카오 지도 접속하기
    searchbox = driver.find_element_by_xpath("//input[@id='search.keyword.query']") # 검색창에 입력하기
    searchbox.send_keys(item)
    time.sleep(2)
    searchbutton = driver.find_element_by_xpath("//button[@id='search.keyword.submit']") # 검색버튼 누르기
    driver.execute_script("arguments[0].click();", searchbutton)
    time.sleep(2)
    
    if len(driver.find_elements_by_xpath("//a[@class='moreview']")) != 0:
        print('식당 존재')
        driver.execute_script('window.open("about:blank", "_blank");') # 새 탭 열기
        reviewbutton = driver.find_element_by_xpath("//a[@class='numberofscore']")
        time.sleep(2)
        content_url = reviewbutton.get_attribute("href") 
        #상세보기 URL수집
        url = driver.find_element_by_xpath('//*[@id="info.search.place.list"]/li[1]/div[5]/div[4]/a[1]').get_attribute('href')
        data.iloc[indexing,-3] = url
        print(url)
        tabs = driver.window_handles
        driver.switch_to_window(tabs[1]) # 새 탭으로 이동
        driver.get(content_url) # 링크 열기
        time.sleep(3)
        html = driver.page_source
        time.sleep(20)
        soup = BeautifulSoup(html, 'html.parser')
        review_lists = soup.select('.list_evaluation > li')
        print(len(review_lists))
        if len(review_lists) != 0 :
            for i, review in enumerate(review_lists) :
                user_review = review.select('.txt_comment > span') # 리뷰
                rating = review.select('.grade_star > em') # 별점
                img_url = review.select('.link_photo > img') #이미지url
                user_id = review.select('.append_item > a[data-userid]') # user 정보 html 파싱
                timestamp = review.select(' div > span.time_write') #시간정보
                try:
                    row = {"UserID":user_id[0].get('data-userid'),"ItemID":item,"Rating":rating[0].text,"Timestamp":timestamp[0].text}
                    row = pd.DataFrame(row, index=[i])
                    rating_df = rating_df.append(row,ignore_index=True)
                    review_row = {"ItemID" : item, "review" : user_review[0].text}
                    review_row = pd.DataFrame(review_row, index=[i])
                    review_elem = review_elem.append(review_row, ignore_index = True)
                    try :
                        img_row = {"ItemID" : item, "img_url" : img_ur}
                        img_row = pd.DataFrame(img_row, index=[i])
                        img_elem = img_elem.append(img_row, ignore_index=True)
                    except :
                        img_row = {"ItemID" : item, "img_url" : None}
                        img_row = pd.DataFrame(img_row, index=[i])
                        img_elem = img_elem.append(img_row, ignore_index=True)

                    time.sleep(1)
                except:
                    row = {"UserID":None,"ItemID":item,"Rating":None,"Timestamp":timestamp[0].text}
                    row = pd.DataFrame(row, index=[i])
                    rating_df = rating_df.append(row,ignore_index=True)
                    review_row = {"ItemID" : item, "review" : user_review[0].text}
                    review_row = pd.DataFrame(review_row, index=[i])
                    review_elem = review_elem.append(review_row, ignore_index = True)
                    try :
                        img_row = {"ItemID" : item, "img_url" : img_url}
                        img_row = pd.DataFrame(img_row, index=[i])
                        img_elem = img_elem.append(img_row, ignore_index=True)
                    except :
                        img_row = {"ItemID" : item, "img_url" : None}
                        img_row = pd.DataFrame(img_row, index=[i])
                        img_elem = img_elem.append(img_row, ignore_index=True)
                    time.sleep(1)

                
                    
        else :
            print("리뷰가 없습니다")
            
        try:
            for i in range(2,500):
                time.sleep(3)
                another_review = driver.find_element_by_xpath("//a[@data-page='" + str(i) + "']")
                another_review.click()
                time.sleep(3)
                html = driver.page_source
                soup = BeautifulSoup(html, 'html.parser')
                review_lists = soup.select('.list_evaluation > li')
                if len(review_lists) != 0 :
                    for i, review in enumerate(review_lists) :
                        user_review = review.select('.txt_comment > span') # 리뷰
                        rating = review.select('.grade_star > em') # 별점
                        img_url = review.select('.link_photo > img') #이미지url
                        user_id = review.select('.append_item > a[data-userid]') # user 정보 html 파싱
                        timestamp = review.select(' div > span.time_write') #시간정보
                        try:
                            row = {"UserID":user_id[0].get('data-userid'),"ItemID":item,"Rating":rating[0].text,"Timestamp":timestamp[0].text}
                            row = pd.DataFrame(row, index=[i])
                            rating_df = rating_df.append(row,ignore_index=True)
                            review_row = {"ItemID" : item, "review" : user_review[0].text}
                            review_row = pd.DataFrame(review_row, index=[i])
                            review_elem = review_elem.append(review_row, ignore_index = True)
                            try :
                                img_row = {"ItemID" : item, "img_url" : img_url}
                                img_row = pd.DataFrame(img_row, index=[i])
                                img_elem = img_elem.append(img_row, ignore_index=True)
                            except :
                                img_row = {"ItemID" : item, "img_url" : None}
                                img_row = pd.DataFrame(img_row, index=[i])
                                img_elem = img_elem.append(img_row, ignore_index=True)
                            time.sleep(1)
                            
                        except:
                            row = {"UserID":None,"ItemID":item,"Rating":None,"Timestamp":timestamp[0].text}
                            row = pd.DataFrame(row, index=[i])
                            rating_df = rating_df.append(row,ignore_index=True)
                            review_row = {"ItemID" : item, "review" : user_review[0].text}
                            review_row = pd.DataFrame(review_row, index=[i])
                            review_elem = review_elem.append(review_row, ignore_index = True)
                            try :
                                img_row = {"ItemID" : item, "img_url" : img_url}
                                img_row = pd.DataFrame(img_row, index=[i])
                                img_elem = img_elem.append(img_row, ignore_index=True)
                            except :
                                img_row = {"ItemID" : item, "img_url" : None}
                                img_row = pd.DataFrame(img_row, index=[i])
                                img_elem = img_elem.append(img_row, ignore_index=True)
                            time.sleep(1)
                        
                        
                            
        except:
            print("더 이상 리뷰 존재 X")
            driver.close()
        driver.switch_to_window(tabs[0])
        print("기본 페이지로 돌아가자")
            
    else:
        data.iloc[indexing,-3] = None
        print("식당 존재 x")
    
    indexing += 1


진행상황 : 1/1190
식당 존재 x
진행상황 : 2/1190
식당 존재
https://place.map.kakao.com/21539441
0
리뷰가 없습니다
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 3/1190
식당 존재
https://place.map.kakao.com/424982028
1
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 4/1190
식당 존재
https://place.map.kakao.com/737348953
5
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 5/1190
식당 존재
https://place.map.kakao.com/1387845195
0
리뷰가 없습니다
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 6/1190
식당 존재
https://place.map.kakao.com/1624596761
0
리뷰가 없습니다
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 7/1190
식당 존재 x
진행상황 : 8/1190
식당 존재
https://place.map.kakao.com/118064282
1
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 9/1190
식당 존재 x
진행상황 : 10/1190
식당 존재
https://place.map.kakao.com/1638385767
2
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 11/1190
식당 존재
https://place.map.kakao.com/280376044
0
리뷰가 없습니다
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 12/1190
식당 존재
https://place.map.kakao.com/248352144
0
리뷰가 없습니다
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 13/1190
식당 존재
https://place.map.kakao.com/20085479
4
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 14/1190
식당 존재
htt

식당 존재 x
진행상황 : 133/1190
식당 존재 x
진행상황 : 134/1190
식당 존재 x
진행상황 : 135/1190
식당 존재 x
진행상황 : 136/1190
식당 존재 x
진행상황 : 137/1190
식당 존재
https://place.map.kakao.com/8105265
5
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 138/1190
식당 존재 x
진행상황 : 139/1190
식당 존재
https://place.map.kakao.com/22547052
5
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 140/1190
식당 존재 x
진행상황 : 141/1190
식당 존재
https://place.map.kakao.com/24637490
5
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 142/1190
식당 존재 x
진행상황 : 143/1190
식당 존재 x
진행상황 : 144/1190
식당 존재 x
진행상황 : 145/1190
식당 존재
https://place.map.kakao.com/26881063
4
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 146/1190
식당 존재 x
진행상황 : 147/1190
식당 존재 x
진행상황 : 148/1190
식당 존재
https://place.map.kakao.com/1677011282
1
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 149/1190
식당 존재 x
진행상황 : 150/1190
식당 존재 x
진행상황 : 151/1190
식당 존재 x
진행상황 : 152/1190
식당 존재 x
진행상황 : 153/1190
식당 존재
https://place.map.kakao.com/1377657605
5
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 154/1190
식당 존재 x
진행상황 : 155/1190
식당 존재
https://place.map.kakao.com/18518683
2
더 이상 리뷰 존재 X
기본 페이지로 돌

더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 265/1190
식당 존재
https://place.map.kakao.com/22321537
0
리뷰가 없습니다
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 266/1190
식당 존재
https://place.map.kakao.com/1191547549
0
리뷰가 없습니다
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 267/1190
식당 존재 x
진행상황 : 268/1190
식당 존재
https://place.map.kakao.com/26419830
0
리뷰가 없습니다
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 269/1190
식당 존재
https://place.map.kakao.com/129539964
3
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 270/1190
식당 존재
https://place.map.kakao.com/502547128
3
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 271/1190
식당 존재 x
진행상황 : 272/1190
식당 존재 x
진행상황 : 273/1190
식당 존재 x
진행상황 : 274/1190
식당 존재 x
진행상황 : 275/1190
식당 존재
https://place.map.kakao.com/458571814
3
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 276/1190
식당 존재
https://place.map.kakao.com/7961275
5
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 277/1190
식당 존재
https://place.map.kakao.com/1244156690
4
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 278/1190
식당 존재
https://place.map.kakao.com/2130845086
1
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 279/1190
식당 존재
https://place.

5
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 389/1190
식당 존재
https://place.map.kakao.com/1860681564
5
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 390/1190
식당 존재
https://place.map.kakao.com/1717480374
1
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 391/1190
식당 존재
https://place.map.kakao.com/1106428073
5
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 392/1190
식당 존재
https://place.map.kakao.com/16066885
5
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 393/1190
식당 존재
https://place.map.kakao.com/209798594
5
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 394/1190
식당 존재 x
진행상황 : 395/1190
식당 존재
https://place.map.kakao.com/22728521
4
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 396/1190
식당 존재
https://place.map.kakao.com/830350701
2
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 397/1190
식당 존재 x
진행상황 : 398/1190
식당 존재
https://place.map.kakao.com/733560860
5
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 399/1190
식당 존재 x
진행상황 : 400/1190
식당 존재
https://place.map.kakao.com/12618029
5
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 401/1190
식당 존재
https://place.map.kakao.com/15175183
4
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 402/1190
식당 존재

5
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 520/1190
식당 존재 x
진행상황 : 521/1190
식당 존재
https://place.map.kakao.com/388622337
5
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 522/1190
식당 존재
https://place.map.kakao.com/15218998
1
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 523/1190
식당 존재
https://place.map.kakao.com/1232433272
1
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 524/1190
식당 존재
https://place.map.kakao.com/16054353
3
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 525/1190
식당 존재
https://place.map.kakao.com/21331994
1
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 526/1190
식당 존재 x
진행상황 : 527/1190
식당 존재
https://place.map.kakao.com/713295374
0
리뷰가 없습니다
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 528/1190
식당 존재
https://place.map.kakao.com/753592260
5
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 529/1190
식당 존재 x
진행상황 : 530/1190
식당 존재
https://place.map.kakao.com/17165638
5
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 531/1190
식당 존재 x
진행상황 : 532/1190
식당 존재
https://place.map.kakao.com/16417433
0
리뷰가 없습니다
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 533/1190
식당 존재
https://place.map.kakao.com/20133399
5
더 이상 리뷰 존

0
리뷰가 없습니다
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 648/1190
식당 존재
https://place.map.kakao.com/27500873
5
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 649/1190
식당 존재 x
진행상황 : 650/1190
식당 존재
https://place.map.kakao.com/12526369
5
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 651/1190
식당 존재
https://place.map.kakao.com/20944162
0
리뷰가 없습니다
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 652/1190
식당 존재
https://place.map.kakao.com/26532882
4
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 653/1190
식당 존재
https://place.map.kakao.com/211153126
1
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 654/1190
식당 존재
https://place.map.kakao.com/1444989476
0
리뷰가 없습니다
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 655/1190
식당 존재
https://place.map.kakao.com/819657300
5
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 656/1190
식당 존재
https://place.map.kakao.com/24637490
5
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 657/1190
식당 존재 x
진행상황 : 658/1190
식당 존재
https://place.map.kakao.com/1375533844
0
리뷰가 없습니다
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 659/1190
식당 존재
https://place.map.kakao.com/455118989
5
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 660/

식당 존재 x
진행상황 : 773/1190
식당 존재
https://place.map.kakao.com/1440633536
1
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 774/1190
식당 존재 x
진행상황 : 775/1190
식당 존재
https://place.map.kakao.com/668868626
5
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 776/1190
식당 존재
https://place.map.kakao.com/434069808
5
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 777/1190
식당 존재 x
진행상황 : 778/1190
식당 존재
https://place.map.kakao.com/13324869
1
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 779/1190
식당 존재
https://place.map.kakao.com/27335071
5
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 780/1190
식당 존재 x
진행상황 : 781/1190
식당 존재 x
진행상황 : 782/1190
식당 존재 x
진행상황 : 783/1190
식당 존재
https://place.map.kakao.com/24529846
5
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 784/1190
식당 존재
https://place.map.kakao.com/1051451549
2
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 785/1190
식당 존재 x
진행상황 : 786/1190
식당 존재 x
진행상황 : 787/1190
식당 존재
https://place.map.kakao.com/718180010
1
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 788/1190
식당 존재 x
진행상황 : 789/1190
식당 존재
https://place.map.kakao.com/20181276
5
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 790

https://place.map.kakao.com/21398605
0
리뷰가 없습니다
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 900/1190
식당 존재
https://place.map.kakao.com/1932670812
0
리뷰가 없습니다
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 901/1190
식당 존재 x
진행상황 : 902/1190
식당 존재 x
진행상황 : 903/1190
식당 존재
https://place.map.kakao.com/9752530
5
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 904/1190
식당 존재
https://place.map.kakao.com/1514325271
0
리뷰가 없습니다
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 905/1190
식당 존재
https://place.map.kakao.com/24529846
5
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 906/1190
식당 존재
https://place.map.kakao.com/7968405
5
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 907/1190
식당 존재 x
진행상황 : 908/1190
식당 존재
https://place.map.kakao.com/27579564
5
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 909/1190
식당 존재
https://place.map.kakao.com/27606261
3
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 910/1190
식당 존재
https://place.map.kakao.com/1335881809
5
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 911/1190
식당 존재 x
진행상황 : 912/1190
식당 존재 x
진행상황 : 913/1190
식당 존재
https://place.map.kakao.com/877029395
2
더 이상 리뷰 존재 X
기본 페이지로 돌아가자


식당 존재
https://place.map.kakao.com/1241796808
5
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 1015/1190
식당 존재 x
진행상황 : 1016/1190
식당 존재
https://place.map.kakao.com/1897325805
5
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 1017/1190
식당 존재
https://place.map.kakao.com/1860681564
5
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 1018/1190
식당 존재 x
진행상황 : 1019/1190
식당 존재
https://place.map.kakao.com/1090931782
5
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 1020/1190
식당 존재
https://place.map.kakao.com/2067214806
5
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 1021/1190
식당 존재
https://place.map.kakao.com/1030119391
5
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 1022/1190
식당 존재 x
진행상황 : 1023/1190
식당 존재 x
진행상황 : 1024/1190
식당 존재
https://place.map.kakao.com/273705133
5
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 1025/1190
식당 존재
https://place.map.kakao.com/587873402
5
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 1026/1190
식당 존재 x
진행상황 : 1027/1190
식당 존재
https://place.map.kakao.com/837824252
5
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 1028/1190
식당 존재
https://place.map.kakao.com/2024611389
4
더 이상 리뷰 존재 X
기본 페이지로 

https://place.map.kakao.com/1144986957
5
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 1128/1190
식당 존재
https://place.map.kakao.com/1528712525
4
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 1129/1190
식당 존재
https://place.map.kakao.com/1268657322
0
리뷰가 없습니다
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 1130/1190
식당 존재
https://place.map.kakao.com/1752052555
4
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 1131/1190
식당 존재 x
진행상황 : 1132/1190
식당 존재 x
진행상황 : 1133/1190
식당 존재 x
진행상황 : 1134/1190
식당 존재 x
진행상황 : 1135/1190
식당 존재
https://place.map.kakao.com/874421812
5
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 1136/1190
식당 존재 x
진행상황 : 1137/1190
식당 존재
https://place.map.kakao.com/18076508
5
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 1138/1190
식당 존재
https://place.map.kakao.com/1381226091
5
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 1139/1190
식당 존재
https://place.map.kakao.com/796762892
0
리뷰가 없습니다
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 1140/1190
식당 존재
https://place.map.kakao.com/427257117
4
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 1141/1190
식당 존재 x
진행상황 : 1142/1190
식당 존재
https://place.map.kakao.com

In [7]:
rating_df

,UserID,ItemID,Rating,Timestamp
0,tvvope,삼성중앙역 르플로우,5점,2019.11.09.
1,16mn981,신논현역 바나프레소,5점,2021.05.14.
2,vurkuq,신논현역 바나프레소,3점,2021.04.22.
3,1iqf80p,신논현역 바나프레소,5점,2021.04.22.
4,10etitd,신논현역 바나프레소,2점,2021.02.19.
...,...,...,...,...
4490,l1t2jt,마곡나루역 카페,5점,2020.10.13.
4491,vsioki,마곡나루역 카페,4점,2020.09.26.
4492,sd1dqt,마곡나루역 카페,5점,2020.09.13.
4493,170ma5h,마곡나루역 카페,3점,2020.03.15.


In [8]:
img_elem

,ItemID,img_url
0,삼성중앙역 르플로우,None
1,신논현역 바나프레소,None
2,신논현역 바나프레소,None
3,신논현역 바나프레소,None
4,신논현역 바나프레소,None
...,...,...
4490,마곡나루역 카페,None
4491,마곡나루역 카페,None
4492,마곡나루역 카페,None
4493,마곡나루역 카페,None


In [9]:
data

,상호명,상권업종중분류명,상권업종소분류명,도로명주소,역명,거리,카카오url,역,검색어
7,빈트리망원점,커피점/카페,커피전문점/카페/다방,서울특별시 마포구 망원로 22,9호선 선유도,1294.649872,NaN,선유도,선유도역 빈트리망원점
13,더블유카페,커피점/카페,커피전문점/카페/다방,서울특별시 강남구 언주로 407,9호선 선정릉,764.611819,https://place.map.kakao.com/21539441,선정릉,선정릉역 더블유카페
15,커피빈,커피점/카페,커피전문점/카페/다방,서울특별시 강남구 테헤란로 335,9호선 삼성중앙,461.237983,https://place.map.kakao.com/424982028,삼성중앙,삼성중앙역 커피빈
29,카페그리다,커피점/카페,커피전문점/카페/다방,서울특별시 강남구 테헤란로69길 14,9호선 삼성중앙,554.451907,https://place.map.kakao.com/737348953,삼성중앙,삼성중앙역 카페그리다
35,홈플러스어라운드홈,커피점/카페,커피전문점/카페/다방,서울특별시 강서구 화곡로 398,9호선 가양,236.335955,https://place.map.kakao.com/1387845195,가양,가양역 홈플러스어라운드홈
...,...,...,...,...,...,...,...,...,...
12774,티컬렉티브,커피점/카페,커피전문점/카페/다방,서울특별시 강남구 봉은사로 449,9호선 삼성중앙,35.645121,NaN,삼성중앙,삼성중앙역 티컬렉티브
12782,트윗젤,커피점/카페,커피전문점/카페/다방,서울특별시 강남구 테헤란로 340,9호선 삼성중앙,478.709107,NaN,삼성중앙,삼성중앙역 트윗젤
12784,후르츠13.5,커피점/카페,커피전문점/카페/다방,서울특별시 강서구 마곡서로 158,9호선 마곡나루,235.823443,NaN,마곡나루,마곡나루역 후르츠13.5
12785,카페,커피점/카페,커피전문점/카페/다방,서울특별시 강서구 마곡중앙로 165,9호선 마곡나루,135.140328,NaN,마곡나루,마곡나루역 카페


In [10]:
img_elem

,ItemID,img_url
0,삼성중앙역 르플로우,None
1,신논현역 바나프레소,None
2,신논현역 바나프레소,None
3,신논현역 바나프레소,None
4,신논현역 바나프레소,None
...,...,...
4490,마곡나루역 카페,None
4491,마곡나루역 카페,None
4492,마곡나루역 카페,None
4493,마곡나루역 카페,None


In [11]:
rating_df

,UserID,ItemID,Rating,Timestamp
0,tvvope,삼성중앙역 르플로우,5점,2019.11.09.
1,16mn981,신논현역 바나프레소,5점,2021.05.14.
2,vurkuq,신논현역 바나프레소,3점,2021.04.22.
3,1iqf80p,신논현역 바나프레소,5점,2021.04.22.
4,10etitd,신논현역 바나프레소,2점,2021.02.19.
...,...,...,...,...
4490,l1t2jt,마곡나루역 카페,5점,2020.10.13.
4491,vsioki,마곡나루역 카페,4점,2020.09.26.
4492,sd1dqt,마곡나루역 카페,5점,2020.09.13.
4493,170ma5h,마곡나루역 카페,3점,2020.03.15.


In [12]:
review_elem

,ItemID,review
0,삼성중앙역 르플로우,꽃도 잘만들어주셔서 이쁘고 사장님도 정말정말 친절해요~ 만족!
1,신논현역 바나프레소,강남역 갈 때 꼭 들리는 카페 .. 가성비 카페하면 바나프레소인데 강남점은 작업할 ...
2,신논현역 바나프레소,달달한게 설탕이 많네요
3,신논현역 바나프레소,싼마이
4,신논현역 바나프레소,"와이파이!!!가 너무 느려요ㅜ 음료는 가격이 너무 착한데. 대부분 2,000원에서 ..."
...,...,...
4490,마곡나루역 카페,롤케이크 진짜 맛있어요
4491,마곡나루역 카페,굿굿
4492,마곡나루역 카페,아이스크림 맛있어요 공간도 넓어서 좋네요.
4493,마곡나루역 카페,모양은 이쁜데 맛은 쏘쏘함 포장해왔는데 해동이 덜됐는지 얼음맛이났어요


In [13]:
img_elem.head()

,ItemID,img_url
0,삼성중앙역 르플로우,None
1,신논현역 바나프레소,None
2,신논현역 바나프레소,None
3,신논현역 바나프레소,None
4,신논현역 바나프레소,None


In [14]:
ItemID_elem = rating_df['ItemID'].unique()

for k in range(len(ItemID_elem)) :
    User_ID_elem = rating_df.loc[rating_df['ItemID'] == ItemID_elem[k]]
    Uder_ID_elem = User_ID_elem[['UserID','ItemID']]
print(User_ID_elem)





ItemID_elem = img_elem['ItemID'].unique()
for j in range(len(ItemID_elem)) :
    item_img_elem = img_elem.loc[img_elem['ItemID'] == ItemID_elem[j]]
print(item_img_elem)



ItemID_elem = review_elem['ItemID'].unique()
for j in range(len(ItemID_elem)) :
    item_review_elem = review_elem.loc[review_elem['ItemID'] == ItemID_elem[j]]
print(item_review_elem)




     UserID     ItemID Rating    Timestamp
4494         노량진역 카페시모     5점  2018.06.22.
         ItemID img_url
4494  노량진역 카페시모    None
         ItemID                 review
4494  노량진역 카페시모  커피도 맛있고 엄청 친절하십니다!!추춴


In [15]:
rating_df.to_csv('C:/Users/정예림/Desktop/추천시스템 프로젝트/data/line9_cafe(final).csv')

In [16]:
User_ID_elem.to_csv('C:/Users/정예림/Desktop/추천시스템 프로젝트/data/line9_cafe_User(1final).csv')
item_img_elem.to_csv('C:/Users/정예림/Desktop/추천시스템 프로젝트/data/line9_cafe_img(final).csv')
item_review_elem.to_csv('C:/Users/정예림/Desktop/추천시스템 프로젝트/data/lline9_cafe_review(final).csv')

In [17]:
ItemID_elem = rating_df['ItemID'].unique()
len(ItemID_elem)


470

In [18]:
data.to_csv('C:/Users/정예림/Desktop/추천시스템 프로젝트/data/line9(final)url.csv')